In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

In [2]:
# path = '/home/hassan/lily/MLA/FDA/inpatient_full_simple.csv'
# inpatient = pd.read_csv(path).iloc[:,1:]
# inpatient.head()

In [2]:
path = '/data/public/MLA/VCHAMPS-Quality/procedures_qual.csv'
procedures = pd.read_csv(path).iloc[:,1:]
procedures

,Internalpatientid,Age at procedure,Procedure date,Procedure code,Procedure code description,State
0,100012,52.100619,2001-08-06 18:37:42.0,99243,OFFICE CONSULTATION FOR A NEW OR ESTABLISHED P...,Texas
1,100012,52.397433,2001-11-23 06:16:51.0,99282,EMERGENCY DEPARTMENT VISIT FOR THE EVALUATION ...,Texas
2,100012,52.594739,2002-02-03 09:02:55.0,99090,ANALYSIS OF CLINICAL DATA STORED IN COMPUTERS ...,New Mexico
3,100012,52.648080,2002-02-22 20:57:17.0,99412,PREVENTIVE MEDICINE COUNSELING AND/OR RISK FAC...,New Mexico
4,100012,52.822460,2002-04-27 14:37:10.0,99204,OFFICE OR OTHER OUTPATIENT VISIT FOR THE EVALU...,New Mexico
...,...,...,...,...,...,...
774430,99941,76.764043,2019-05-01 06:11:00.0,92012,OPHTHALMOLOGICAL SERVICES: MEDICAL EXAMINATION...,Maine
774431,99944,77.937531,2010-06-16 04:01:12.0,99070,"SUPPLIES AND MATERIALS (EXCEPT SPECTACLES), PR...",Massachusetts
774432,99944,77.937531,2010-06-16 04:01:12.0,99201,OFFICE OR OTHER OUTPATIENT VISIT FOR THE EVALU...,Massachusetts
774433,99944,80.746340,2013-04-07 18:53:35.0,G0154,DIRECT SKILLED NURSING SERVICES OF A LICENSED ...,Massachusetts


In [3]:
procedures = procedures.drop_duplicates(subset = ['Internalpatientid', 'Procedure date','Procedure code description'],keep = 'first')

In [9]:
# inpatient_date = inpatient[['Internalpatientid','Discharge date']]
# inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
# inpatient_date

In [10]:
# # Filter procedure date that's before discharge date
# inpatient_pro = procedures.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
# inpatient_pro2 = inpatient_pro.loc[inpatient_pro['Procedure date'] <= inpatient_pro['Discharge date']]
# inpatient_pro2 

In [4]:
# Calculate total procedures before last admission for each patient
procedures['total_procedure'] = procedures['Procedure code description'].groupby(procedures['Internalpatientid']).transform('count')

/tmp/ipykernel_278536/750668648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  procedures['total_procedure'] = procedures['Procedure code description'].groupby(procedures['Internalpatientid']).transform('count')


In [5]:
# Calculate total procedures that's surgery before last admission for each patient
surgery_pro = procedures.loc[procedures['Procedure code description'].str.contains('SURG')]
surgery_pro['num_surgery_pro'] = surgery_pro['Procedure code'].groupby(surgery_pro['Internalpatientid']).transform('count')

/tmp/ipykernel_278536/3501479404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surgery_pro['num_surgery_pro'] = surgery_pro['Procedure code'].groupby(surgery_pro['Internalpatientid']).transform('count')


In [6]:
procedures['Internalpatientid'].nunique()

1000

In [7]:
columns = ['Internalpatientid', 'total_procedure']
df = procedures[columns]
df = df.groupby('Internalpatientid').mean().reset_index(drop = False)
df = df.merge(surgery_pro[['Internalpatientid','num_surgery_pro']], how = 'left',on = 'Internalpatientid')
df['num_surgery_pro'] = df['num_surgery_pro'].fillna(0)
df = df.groupby('Internalpatientid').mean().reset_index(drop = False)
df

,Internalpatientid,total_procedure,num_surgery_pro
0,67,691.0,0.0
1,200,1971.0,51.0
2,291,504.0,0.0
3,330,1353.0,16.0
4,351,350.0,1.0
...,...,...,...
995,167907,188.0,0.0
996,167917,384.0,0.0
997,168008,916.0,3.0
998,168496,241.0,3.0


In [8]:
#df.to_csv('/home/daisy/FDA_Dataset/inpatient_procedures_clean_cvd_mortality_qaulity.csv')